In [ ]:
#This Notebook Analyzes  Ansible Using ChatGPT-3 Turbo with Performance Metrics

In [ ]:
!pip install --upgrade openai


  Using cached openai-1.68.2-py3-none-any.whl.metadata (25 kB)
Using cached openai-1.68.2-py3-none-any.whl (606 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install jsonformer

  Attempting uninstall: termcolor
    Found existing installation: termcolor 3.1.0
    Uninstalling termcolor-3.1.0:
      Successfully uninstalled termcolor-3.1.0


In [ ]:
!pip show openai

Name: openai
Version: 1.68.2
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = 'gpt2'  # This is GPT-2, other variants are available, e.g., gpt2-medium
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Encode the prompt text
input_text = "Write a creative story about a unicorn."
inputs = tokenizer.encode(input_text, return_tensors='pt')

# Generate output
outputs = model.generate(inputs, max_length=100, num_return_sequences=1)

# Decode and print the result
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Write a creative story about a unicorn.

The unicorn is a creature that lives in the world. It is a creature that is a part of the world. It is a creature that is a part of the world. It is a creature that is a part of the world. It is a creature that is a part of the world. It is a creature that is a part of the world. It is a creature that is a part of the world. It is a creature that is a


In [ ]:
!pip show transformers

Name: transformers
Version: 4.49.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: coqui-tts, peft, sentence-transformers


In [ ]:
print(df.columns.tolist())  # Debugging: Check actual column names

['Script', 'Label ', 'MisconfigurationFound', 'LLM_Found_Label']


In [ ]:


#ChatGPT-3 Turbo for the Ansible dataset  "
import openai
import asyncio
import pandas as pd
import csv
import json
import os
import time
import datetime

# Set your OpenAI API key
client = openai.OpenAI(api_key="")

# Function to create a structured prompt including the ID
def create_prompt(code_snippet):
    return f"""
    You are a **security expert specializing in Ansible configuration security**.
    Your task is to analyze the following Ansible code and identify **only critical security misconfigurations** — issues that could lead to vulnerabilities, exploitation, or CVEs.

    **Ansible Script to Analyze**:
    ```Ansible
    {code_snippet}
    ```

    ### **Response Format**:
    Return the response in **valid JSON format ONLY** (no extra text):

    - If there are security misconfigurations, return:
    {{
      "issues_found": [
        {{
          "misconfigured_snippet": "<Describe the issue if critical and has cve  otherwise 'N/A'>",
          "related_cve": "<CVE-ID of vulnerabilities caused by misconfigured_snippet if known, otherwise 'N/A'>"
        }},
        {{
          "misconfigured_snippet": "<Describe the issue if critical and has cve  otherwise 'N/A'>",
          "related_cve": "<CVE-ID of vulnerabilities caused by misconfigured_snippet if known, otherwise 'N/A'>"
        }}
      ],
      "misconfiguration_label": 1
    }}

    - If the script is **secure or only has minor issues**, return:
    {{
      "issues_found": [],
      "misconfiguration_label": 0
    }}

    ### **Important Rules**:
      - Focus ONLY on critical issues that can cause ** critical security vulnerabilities**.
      - For each issue:
        - Provide a brief description in `"misconfigured_snippet"`.
        - Attach a `"related_cve"`: real CVE ID if known, otherwise `"N/A"`.
      - Do NOT add explanations or extra text.
      - `"issues_found"` must be an **empty list** if there are **no critical issues**.
    """


# Function to analyze Ansible misconfiguration
async def send_to_openai(prompt, request_num):
    """
    Send the prompt to OpenAI and return extracted security issues, misconfiguration label, and CVEs separately,
    with each misconfigured snippet paired with its corresponding CVE in separate rows.
    """
    while True:
        try:
            # Send request to OpenAI API
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",  # Adjust to GPT-4 if necessary
                messages=[{"role": "user", "content": prompt}],
                max_tokens=4000,
                temperature=0.2,
                top_p=0.4,
            )

            response_text = response.choices[0].message.content.strip()
            print(f"🔹 GPT Response: {response_text}")  # Debugging output

            # Clean the response to extract JSON part
            try:
                start_index = response_text.index('{')
                end_index = response_text.rindex('}') + 1
                json_part = response_text[start_index:end_index]

                # Parse the JSON part
                response_json = json.loads(json_part)
                print(f"🔹 Parsed JSON: {response_json}")  # Debugging output

                issues_list = response_json.get("issues_found", [])
                misconfiguration_label = response_json.get("misconfiguration_label", 0)

                # Ensure misconfiguration_label is valid
                if not isinstance(misconfiguration_label, int) or misconfiguration_label not in [0, 1]:
                    misconfiguration_label = 1 if issues_list else 0

                # Extract issues (return each issue with its corresponding CVE as separate rows)
                extracted_issues_and_cves = []

                for issue in issues_list:
                    # Extract each misconfigured snippet
                    misconfigured_snippet = issue.get("misconfigured_snippet", "No snippet found")

                    # Extract CVEs related to the misconfiguration
                    cves = issue.get("related_cve", "N/A")

                    # Handle cases where there is more than one CVE (either list or string)
                    if isinstance(cves, str):
                        extracted_issues_and_cves.append((misconfigured_snippet, cves, misconfiguration_label))  # Include label
                    elif isinstance(cves, list):
                        for cve in cves:
                            extracted_issues_and_cves.append((misconfigured_snippet, cve, misconfiguration_label))  # Include label

                print(f"🔹 Extracted Issues and CVEs: {extracted_issues_and_cves}")  # Debugging output

                # Return the extracted issues, their corresponding CVEs, and the misconfiguration label
                return extracted_issues_and_cves

            except (ValueError, json.JSONDecodeError) as e:
                print(f"❌ JSON Error: {str(e)}. Defaulting to 'No issue found'.")
                return [], 0

        except Exception as e:
            print(f"❌ Request failed: {str(e)}. Retrying in 2 seconds...")
            await asyncio.sleep(2)
            continue
async def process_and_save_results(input_csv, output_csv):
    # Read the input CSV using pandas
    df = pd.read_csv(input_csv)

    # Open the output CSV to write results
    with open(output_csv, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)

        # Write headers in the output CSV
        writer.writerow([
            "Row_ID",
            "Fixing_Commit_Code", "Bug_Inducing_Commit_Code",
            "Misconfiguration_Label",
            "Snippet_Type",  # New column for "Fix" or "Inducing"
            "Misconfigured_Snippet", "CVE"
        ])

        # Process each row in the input CSV
        for i, row in df.iterrows():
            # Extract the commit codes from the CSV
            fixing_commit_code = row["Added_lines_fixing_commit"]
            bug_inducing_commit_code = row["Added_lines_bug_inducing_commit"]

            # Generate the prompts for both fixing and bug-inducing commits
            prompt_fixing = create_prompt(fixing_commit_code)
            prompt_bug = create_prompt(bug_inducing_commit_code)

            # Send the prompts to OpenAI and get the extracted issues and CVEs
            await asyncio.sleep(0.65)
            extracted_issues_and_cves_fixing = await send_to_openai(prompt_fixing, i)
            await asyncio.sleep(0.65)
            extracted_issues_and_cves_bug = await send_to_openai(prompt_bug, i)

            # For fixing commit, write results to the CSV
            for snippet, cve, label in extracted_issues_and_cves_fixing:
                writer.writerow([i + 1, fixing_commit_code, bug_inducing_commit_code, label, "Fix", snippet, cve])

            # For bug-inducing commit, write results to the CSV
            for snippet, cve, label in extracted_issues_and_cves_bug:
                writer.writerow([i + 1, fixing_commit_code, bug_inducing_commit_code, label, "Inducing", snippet, cve])

    # Print a message when done
    print(f"\n✅ Analysis complete! Results saved in '{output_csv}'")

# Define input and output CSV file paths
input_csv = "/content/drive/My Drive/expermintstarcoder/imp/merged_file_ansible.csv"
output_csv = "/content/drive/My Drive/expermintstarcoder/imp/merged_file_ansible_defect_output_chatgbt-3.5-model-2.csv"

# Run the function
await process_and_save_results(input_csv, output_csv)
print("\n✅ All scripts analyzed successfully! The script has finished execution.")
exit()

Streaming output truncated to the last 5000 lines.
        }
    ],
    "misconfiguration_label": 1
}
🔹 Parsed JSON: {'issues_found': [{'misconfigured_snippet': "The 'validate_certs: false' option is set in the apt_key task, disabling SSL certificate validation which can lead to man-in-the-middle attacks.", 'related_cve': 'N/A'}], 'misconfiguration_label': 1}
🔹 Extracted Issues and CVEs: [("The 'validate_certs: false' option is set in the apt_key task, disabling SSL certificate validation which can lead to man-in-the-middle attacks.", 'N/A', 1)]
🔹 GPT Response: {
  "issues_found": [
    {
      "misconfigured_snippet": "The use of user input directly in command execution can lead to command injection vulnerabilities.",
      "related_cve": "CVE-2019-3462"
    }
  ],
  "misconfiguration_label": 1
}
🔹 Parsed JSON: {'issues_found': [{'misconfigured_snippet': 'The use of user input directly in command execution can lead to command injection vulnerabilities.', 'related_cve': 'CVE-2019-3462'

In [ ]:
import openai
import asyncio
import pandas as pd
import csv
import json
import os
import re
import time

# Set your OpenAI API key from environment variable
client = openai.OpenAI(api_key="")

# ✅ Prompt generator for Ansible + CWE-based analysis
def create_prompt(code_snippet):
    prompt = f"""
    You are a security expert specializing in Ansible configuration security.
    Your task is to analyze the following Ansible playbook/task and identify only critical security misconfigurations that could lead to vulnerabilities and have a real CWE ID.
    Focus only on the available code and avoid making assumptions about unavailable details.

    Ansible Playbook to Analyze:
    ```yaml
    {code_snippet}
    ```

    Response Format:
    Return the response in **valid JSON format**:

    - If there are critical issues:
    ```json
    {{
      "issues_found": [
        {{
          "misconfigured_snippet": "<actual misconfigured code snippet>",
          "related_cwe": "<CWE-ID>"
        }}
      ],
      "misconfiguration_label": 1
    }}
    ```

    - If there are NO critical issues:
    ```json
    {{
      "issues_found": [],
      "misconfiguration_label": 0
    }}
    ```

    Instructions:
    - Only list real misconfigurations tied to known CWE IDs.
    - Do NOT insert "N/A".
    - Do not include syntax errors.
    - If no critical misconfiguration is found, return an empty issues_found array.
    """
    return prompt

# ✅ Function to send prompt to OpenAI and extract CWE issues
async def send_to_openai(prompt, request_num, max_retries=5):
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=4000,
                temperature=0.2,
                top_p=0.4,
            )

            response_text = response.choices[0].message.content.strip()

            try:
                start_index = response_text.index('{')
                end_index = response_text.rindex('}') + 1
                json_part = response_text[start_index:end_index]
                response_json = json.loads(json_part)

                issues_list = response_json.get("issues_found", [])
                misconfiguration_label = response_json.get("misconfiguration_label", 0)

                if not isinstance(misconfiguration_label, int) or misconfiguration_label not in [0, 1]:
                    misconfiguration_label = 1 if issues_list else 0

                extracted = []
                cwe_pattern = re.compile(r"^CWE-\d+$")

                for issue in issues_list:
                    snippet = issue.get("misconfigured_snippet", "").strip()
                    cwe = issue.get("related_cwe", "").strip()

                    if cwe_pattern.match(cwe):
                        extracted.append((snippet, cwe, misconfiguration_label))

                return extracted

            except (ValueError, json.JSONDecodeError) as e:
                print(f"❌ JSON Parsing Error: {str(e)}\nGPT Response: {response_text}")
                return []

        except Exception as e:
            retry_count += 1
            print(f"❌ Request {request_num} failed (attempt {retry_count}): {str(e)}. Retrying in 2 seconds...")
            await asyncio.sleep(2)

    print(f"❌ Max retries exceeded for request {request_num}.")
    return []

# ✅ Main function to process CSV and write results
async def process_and_save_results(input_csv, output_csv):
    df = pd.read_csv(input_csv)

    with open(output_csv, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            "Row_ID",
            "Fixing_Commit_Code", "Bug_Inducing_Commit_Code",
            "Misconfiguration_Label",
            "Snippet_Type",  # Fix or Inducing
            "Misconfigured_Snippet", "CWE"
        ])

        for i, row in df.iterrows():
            print(f"\n🔍 Processing Row {i + 1}")
            start = time.time()

            fixing_commit_code = row.get("Added_lines_fixing_commit", "")
            bug_inducing_commit_code = row.get("Added_lines_bug_inducing_commit", "")

            prompt_fixing = create_prompt(fixing_commit_code)
            prompt_bug = create_prompt(bug_inducing_commit_code)

            await asyncio.sleep(0.65)
            fixing_results = await send_to_openai(prompt_fixing, i)

            await asyncio.sleep(0.65)
            bug_results = await send_to_openai(prompt_bug, i)

            # Fixing commit results
            if fixing_results:
                print(f"  ✅ Fixing Commit Issues Found ({len(fixing_results)}):")
                for snippet, cwe, label in fixing_results:
                    print(f"    - CWE: {cwe} | Label: {label}\n      Snippet: {snippet[:100]}...")
                    writer.writerow([i + 1, fixing_commit_code, bug_inducing_commit_code, label, "Fix", snippet, cwe])
            else:
                print(f"  ✅ Fixing Commit: No critical issues.")
                writer.writerow([i + 1, fixing_commit_code, bug_inducing_commit_code, 0, "Fix", "", ""])

            # Bug-inducing commit results
            if bug_results:
                print(f"  ✅ Bug-Inducing Commit Issues Found ({len(bug_results)}):")
                for snippet, cwe, label in bug_results:
                    print(f"    - CWE: {cwe} | Label: {label}\n      Snippet: {snippet[:100]}...")
                    writer.writerow([i + 1, fixing_commit_code, bug_inducing_commit_code, label, "Inducing", snippet, cwe])
            else:
                print(f"  ✅ Bug-Inducing Commit: No critical issues.")
                writer.writerow([i + 1, fixing_commit_code, bug_inducing_commit_code, 0, "Inducing", "", ""])

            end = time.time()
            print(f"  ⏱️ Time taken: {end - start:.2f} seconds")

    print(f"\n✅ Analysis complete! Results saved to: {output_csv}")

# ✅ File paths
input_csv = "/content/drive/My Drive/expermintstarcoder/imp/merged_file_ansible.csv"
output_csv = "/content/drive/My Drive/expermintstarcoder/imp/ansible_chatgpt_3.5_cwes.csv"

# ✅ Run the full script
await process_and_save_results(input_csv, output_csv)
print("✅ All rows processed successfully. Script execution complete.")



🔍 Processing Row 1
  ✅ Fixing Commit Issues Found (1):
    - CWE: CWE-78 | Label: 1
      Snippet: command: php occ config:system:set {{ item.name }} --value="{{ item.value }}"...
  ✅ Bug-Inducing Commit Issues Found (2):
    - CWE: CWE-78 | Label: 1
      Snippet: shell: php occ config:system:set {{ item.name }} --value="{{ item.value }}"...
    - CWE: CWE-78 | Label: 1
      Snippet: command: php occ config:system:set {{ item.name }} --value="{{ item.value }}"...
  ⏱️ Time taken: 3.03 seconds

🔍 Processing Row 2
  ✅ Fixing Commit Issues Found (1):
    - CWE: CWE-928 | Label: 1
      Snippet: when: (not _nextcloud_conf.stat.exists) or (_nextcloud_configured.rc is defined and _nextcloud_confi...
  ✅ Bug-Inducing Commit Issues Found (1):
    - CWE: CWE-799 | Label: 1
      Snippet: check_mode: no...
  ⏱️ Time taken: 2.71 seconds

🔍 Processing Row 3
  ✅ Fixing Commit Issues Found (1):
    - CWE: CWE-928 | Label: 1
      Snippet: when: (nc_sudo_installed_result.rc is defined) and (nc_sud

In [ ]:
import pandas as pd
from sklearn.metrics import (
    precision_recall_fscore_support,
    roc_auc_score,
    accuracy_score
)

# Load the dataset
df = pd.read_csv('/content/drive/My Drive/expermintstarcoder/ansible_final/ansible_chatgpt_3.csv')

# Drop rows with missing predictions
df = df.dropna(subset=['verification'])

# Convert predictions to numeric if needed
df['verification'] = pd.to_numeric(df['verification'], errors='coerce')

# ✅ Replace this with the actual column containing true binary labels (0 and 1)
true_label_col = 'Misconfiguration_Label'  # Example, update it to your actual column name

# Drop rows with missing ground truth
df = df.dropna(subset=[true_label_col])
df[true_label_col] = pd.to_numeric(df[true_label_col], errors='coerce')

# Filter rows with valid ground truth values (0 or 1)
df = df[df[true_label_col].isin([0, 1])]

# Split by Snippet_Type
fixing_df = df[df['Snippet_Type'] == 'Fix']
inducing_df = df[df['Snippet_Type'] == 'Inducing']

# Extract ground truth and predictions
true_fixing = fixing_df[true_label_col]
pred_fixing = fixing_df['verification']

true_inducing = inducing_df[true_label_col]
pred_inducing = inducing_df['verification']

# Calculate metrics for Fixing
fixing_precision, fixing_recall, fixing_f1, _ = precision_recall_fscore_support(
    true_fixing,
    pred_fixing.round(),
    average='binary',
    zero_division=0,
    pos_label=1
)
fixing_accuracy = accuracy_score(true_fixing, pred_fixing.round())

# Calculate metrics for Inducing
inducing_precision, inducing_recall, inducing_f1, _ = precision_recall_fscore_support(
    true_inducing,
    pred_inducing.round(),
    average='binary',
    zero_division=0,
    pos_label=1
)
inducing_accuracy = accuracy_score(true_inducing, pred_inducing.round())

# Print metrics
print(f"Fixing → Precision: {fixing_precision:.4f}, Recall: {fixing_recall:.4f}, F1: {fixing_f1:.4f}, Accuracy: {fixing_accuracy:.4f}")
print(f"Inducing → Precision: {inducing_precision:.4f}, Recall: {inducing_recall:.4f}, F1: {inducing_f1:.4f}, Accuracy: {inducing_accuracy:.4f}")

# Calculate overall average scores
overall_precision = (fixing_precision + inducing_precision) / 2
overall_recall = (fixing_recall + inducing_recall) / 2
overall_f1 = (fixing_f1 + inducing_f1) / 2
overall_accuracy = (fixing_accuracy + inducing_accuracy) / 2

print(f"Overall → Precision: {overall_precision:.4f}, Recall: {overall_recall:.4f}, F1: {overall_f1:.4f}, Accuracy: {overall_accuracy:.4f}")

# ✅ AUC Calculation
try:
    if df[true_label_col].nunique() < 2:
        print("⚠️ Cannot calculate AUC: Ground truth has only one class.")
    else:
        auc = roc_auc_score(df[true_label_col], df['verification'])
        print(f"Overall AUC: {auc:.4f}")
except Exception as e:
    print(f"Error calculating AUC: {e}")

Fixing → Precision: 0.9408, Recall: 0.7901, F1: 0.8589, Accuracy: 0.7644
Inducing → Precision: 0.9853, Recall: 0.8111, F1: 0.8898, Accuracy: 0.8065
Overall → Precision: 0.9630, Recall: 0.8006, F1: 0.8743, Accuracy: 0.7855
Overall AUC: 0.6837
